# compute word co-occurence matrix (extra material)

Below are some examples of other choices we might have made to create the word co-occurence matrix. These are not strictly necessary (or even a good idea), but are meant to give a taste for the plethora of choices we have to make while processing text. They also might be useful example code. 

Feel free to skip the following sections upon first reading.

In [1]:
from nltk.tokenize import  word_tokenize

from scipy.sparse import csr_matrix, lil_matrix

from collections import Counter
import itertools

# Iain's code
from save import save_vocabulary, load_vocabulary, save_matrix, load_matrix

In [2]:
sentences_file = '/Users/iaincarmichael/data/word_embed/scotus/sentences/scotus_sentences.txt'

# load senteces from bag of sentences file
with open(sentences_file) as f:
    sentences = [line.strip('\n') for line in f.readlines()]

print 'loaded %d sentences ' % len(sentences)

loaded 272279 sentences 


# code from before

In [3]:


def sentences2word_tok(sentences):
    """
    Tokenize sentences into words using nltk's word_tokenize.
    
    Parameters
    ----------
    sentences: a list of sentences
    
    Output
    -------
    A list of lists. The outer list has the same length as sentences. The inner lists have one word token per entry
    """
    return [[w for w in word_tokenize(s)] for s in sentences]

def sentences2counts(sentences_word_tok):
    """
    Compute the matrix of word co-occurances.
  
    Some code borrowed from from https://stackoverflow.com/questions/42814452/co-occurrence-matrix-from-list-of-words-in-python
    
    Parameters
    ----------
    sentences_word_tok: list of tokenized sentences
    w2i: dictionary mapping words to indices
    
    Output
    ------
    symmetric, csr matrix with word co-occurence counts
    """
    # create vocabulary set
    vocab = set()
    for s in sentences_word_tok:
        vocab = vocab.union(set(s))
    vocab = list(vocab)
    
    # dict mapping words to indices
    w2i = {vocab[i]: for i in range(len(vocab))}
    
    # Get a list of all of the combinations
    expanded = [tuple(itertools.combinations(s, 2)) for s in sentences_word_tok]
    expanded = itertools.chain(*expanded)

    # Sort the combinations so that A,B and B,A are treated the same
    expanded = [tuple(sorted(d)) for d in expanded]

    # count the combinations
    pair_counts = Counter(expanded)

    # construct counts as lil matrix but return them as csr_matrix
    counts = lil_matrix((len(w2i), len(w2i)))
    for p in pair_counts:
        counts[w2i[p[0]], w2i[p[1]]] = pair_counts[p]

    return (counts + counts.T).tocsr(), vocab

In [1]:
# w2i, i2w = vocab_from_sentences(sentences)
# sentences_word_tok = sentences2word_tok(sentences)
# counts = sentences2counts(sentences_word_tok, w2i)

### Sentence lengths

Text data is often messy and we make a lot of assumptions when we process it which will be incorrect some portion of the time (e.g. the sentence tokenizer will fail sometimes). It's a good idea to explore the data and frequently perform sanity checks using basic statistics. 

Below we plot a histogram of sentence lenths (in this case lengh = number of characters). The histogram shows us there is one very long sentence which is likely a failure of the sentence tokenizer.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:

sent_lengths = [len(s) for s in sentences]
plt.hist(sent_lengths, bins=100);
plt.xlabel('sentence lengths')
plt.ylabel('count')
plt.title('histogram of sentence lengths')

print 'longest sentence is %d characters' % max(sent_lengths)

# uncommet to view the very long sentence


In [ ]:
sentences[np.argmax(sent_lengths)]

### look at commonly occuring words and word counts

In [ ]:
# list of all words which appear in the file
word_list = []
for s in sentences:
    word_list.extend(word_tokenize(s))

# counts the number of times each word appears in the file
word_counts = Counter(word_list)


print 'there are %d unique words' % len(word_counts.keys())
print ''
w = 'constitution'
print '%s appears %d times' % (w, word_counts[w])
w = 'certiorari'
print '%s appears %d times' % (w, word_counts[w])


# show the most commonly occuring words
word_counts.most_common()

### alternate word tokenization

Given a sentence `s`, the easiest way to tokenize it into words is to break it up by spaces i.e.

```python
s.split(' ')
```

We, however, are using nltk's `word_tokenize` function

```python
from nltk.tokenize import word_tokenize
word_tokenize(s)
```

The two are pretty similary, but will sometimes give different results. You can explore the difference below.

In addition to nltk and `split`, there are many other word tokenizers (e.g. [Stanford's CoreNLP package](https://github.com/Lynten/stanford-corenlp)) and you can make your own.

In [ ]:
# find indices where split and word_tokenize don't agree
[i for i in range(100) if len(sentences[i].split(" ")) !=  len(word_tokenize(sentences[i]))]

In [ ]:
s = sentences[52]

print 'sentence:'
print s
print
print 'using s.split(' ')'
print s.split(" ")
print
print 'using word_tokenize from nltk'
print word_tokenize(s)

### bi-grams

Bi-grams are ordered pairs of words i.e `the_quick`, `quick_brown`. We currently are currently using single words as tokens, but we might want to use bi-grams (or higher order n-grams).

In [ ]:
from nltk import bigrams

# we could count bi-grams as follows, but we won't pursue this for now
ex_sentence = 'the quick brown fox jumped over the lazy dog'

print ex_sentence
print
print list(bigrams(word_tokenize(ex_sentence)))

### restrict vocabulary as part of pre-processing

We currently use every word that appears in our sentences. We might want to restrict ourselves to the K (e.g. 1000, 10,000, etc) most commonly ocuring words as a part of pre-processing using only two minor modifications.


This line finds the 1000 most commonly occuring words
```python
word_counts.most_common()[:1000]
```

Given an vocabulary set (e.g. the most commonly occuring words) this line removes all non-vocabulary words
```python
[[w for w in word_tokenize(s) if w in vocab] for s in sentences]
```

In [ ]:
common_words = word_counts.most_common()[:1000]

# new vocabulary
vocab = set([cw[0] for cw in common_words])

In [ ]:
# remove words that are not in the vocabulary while tokeinzing into sentences
sentences_word_tok = [[w for w in word_tokenize(s) if w in vocab] for s in sentences]

### remove short sentences

We can remove very short sentences (e.g. those with 2 or fewer words) as follows

In [ ]:
# remove sentences that have less than three words
sentences_word_tok = [s for s in sentences_word_tok if len(s) >= 3]

### stemming

Stemming converts words into their base form in some sense (read more about it here http://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization). For example, we might not want to distinguish between **hat** and **hats**. Stemming would convert **hats** into **hat**.

We could have stemmed all of our words as a first pre-processing step. nltk provides a number of stemmers including the Porter and Snowball stemmer

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

s = sentences_word_tok[10]

print s
print
port_stemmer = PorterStemmer()
print [port_stemmer.stem(w) for w in s]

sb_stemmer = SnowballStemmer(language='english')
print [sb_stemmer.stem(w) for w in s]

### naive sentence to counts

The most straightforward way to count word co-occurences within a sentence is with a double for loop (see below). While the code is easier to understand, than the above `sentences2counts()`, it runs much slower.

In [ ]:
def sentences2counts_naive(sentences_word_tok, w2i):
    """
    Compute the matrix of word co-occurances.
    This is the most simple way of counting co-occurences, but will take a long time to run
    
    Parameters
    ----------
    sentences_word_tok: list of tokenized sentences
    w2i: dictionary mapping words to indices
    
    Output
    ------
    symmetric, csr matrix with word co-occurence counts
    """

    # construct counts as lil matrix but return them as csr_matrix
    counts = lil_matrix((len(w2i), len(w2i)))

    # for each sentence, go through all word pairs and update counts
    for s in sentences_word_tok:
        for i in range(len(s)):
            x = s[i]
            for j in range(i + 1, len(s)):
                y = s[j]

                counts[w2i[x], w2i[y]] = counts[w2i[x], w2i[y]] + 1

    counts = counts + counts.T
    return counts.tocsr()

In [ ]:
print 'naive sentence2counts'
%time _ = sentences2counts_naive(sentences_word_tok, w2i)
print
print 'smarter sentence2counts'
%time _ = sentences2counts(sentences_word_tok, w2i)